In [ ]:
import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import MO
from prophet import Prophet
import numpy as np
import os

In [ ]:
# Lectura de serie de tiempo
os.chdir("/home/cesar/Documentos/Banco_Base")
data = pd.read_csv('./Ejercicio_2/datasets/tipo_cambio.csv')
data['Fecha'] = pd.to_datetime(data['Fecha'],format="%d/%m/%Y")

In [ ]:
# Lectura de precio de crudo de petroleo
petroleo = pd.read_csv('./Ejercicio_2/datasets/petroleo.csv')
petroleo['precio_dolares'] = petroleo['precio_dolares'].replace('N/E', np.nan)
petroleo["precio_dolares"] = pd.to_numeric(petroleo["precio_dolares"])
petroleo['precio_dolares'] = petroleo['precio_dolares'].interpolate(method='linear')
petroleo['Fecha'] = pd.to_datetime(petroleo['Fecha'],format="%d/%m/%Y")
# Merge de datos
data2 = pd.merge(data, petroleo, on='Fecha', how='left')
data2_valido = data2[data2['Fecha'] <= pd.to_datetime('2024-09-30')]
data2_no_valido= data2[data2['Fecha'] > pd.to_datetime('2024-09-30')]

data2_valido.columns = ['ds', 'y', 'precio_petroleo']  # Prophet usa columnas 'ds' (fecha) y 'y' (valor)
data2_valido



In [ ]:
### Crear un calendario de días festivos en México
class DiasFestivos(AbstractHolidayCalendar):
    rules = [
        Holiday('Año Nuevo', month=1, day=1),
        # Primer lunes de febrero
        Holiday('Día de la Constitución', month=2, day=1, offset=DateOffset(weekday=MO(1))),
        # Tercer lunes de marzo
        Holiday('Natalicio de Benito Juárez', month=3, day=1, offset=DateOffset(weekday=MO(3))),
        Holiday('Día del Trabajo', month=5, day=1),
        Holiday('Día de la Independencia', month=9, day=16),
        # Tercer lunes de noviembre
        Holiday('Día de la Revolución', month=11, day=1, offset=DateOffset(weekday=MO(3))),
        Holiday('Navidad', month=12, day=25),
    ]

# Generar las fechas festivas
calendar = DiasFestivos()
festivos_mx = calendar.holidays(start='2014-01-02', end='2024-12-31')
festivos_mx


In [ ]:
###### Ajuste de modelo Prophet
# Modelo sin variables externas
model = Prophet(
    holidays=pd.DataFrame({'holiday': 'Festivos_MX', 'ds': festivos_mx}),
    daily_seasonality=False,
    weekly_seasonality=True,  
    yearly_seasonality=True 
)
model.fit(data2_valido.drop(['precio_petroleo'], axis=1))

# Modelo con variables externas
model2 = Prophet(
    holidays=pd.DataFrame({'holiday': 'Festivos_MX', 'ds': festivos_mx}),
    daily_seasonality=False,
    weekly_seasonality=True,  
    yearly_seasonality=True 
)
model2.add_regressor('precio_petroleo')
model2.fit(data2_valido)

In [ ]:
### Generar el pronóstico y MAPE para model 1
# Dataframe de pronostico
future = model.make_future_dataframe(periods=58)  # Pronosticar hasta 2024-11-27
future = future[~future['ds'].dt.weekday.isin([5, 6])]  # Excluir fines de semana
future = future[~future['ds'].isin(festivos_mx)]       # Excluir días festivos
# Pronostico
forecast = model.predict(future)
df_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
df_test = df_forecast.tail(40)
# Calculo del mape
pe = abs(np.array(df_test['yhat']) - np.array(data2_no_valido['Pesos_por_dolar'])) / np.array(data2_no_valido['Pesos_por_dolar'])
mape = np.mean(pe)
mape


In [ ]:
# model.plot(forecast)
# model.plot_components(forecast)
data2_no_valido